
# Feature Scaling


The purpose of this is to bring features to the same scales. Assume we have 2 feature x1 and x2 in ranges 1 to 10, and 1 to 200,000, respectively. The features are in different scales and the largest feautre x2 will dominate. Feature scaling brings features to the same scale. 

There are 2 options which are: Normalization and Standardization.

Note: Standardization is recommended to improve learning of ML algorithms during gradient descent. 

Run this model on the web at: http://www.rcalix.com/projects/CIVSDOE/CFDallDataNorm/index.html

## Normalization

$ \large  x_{norm} = \frac {(x - x_{min})}{(x_{max} - x_{min})}  $

## Standardization

$  \large x_{stand} = \frac {(x - \mu _x)}{\sigma _x}   $




## Experiment description

This run is done using CFD. Used an 80% train test split. 
The CFD data has 56 samples where each sample has 27 features. There are 10 inputs and 17 outputs. All inputs and outputs were used. 

Tested various DL architectures.  


Train_X.........(44, 10)

Train_y.........(44, 17)

Test_X.........(12, 10)

Test_y.........(12, 17)

Results are at the bottom. 



In [1]:
######################################################
##
## regression with scaling and DL for system control
##
######################################################

import torch
import numpy as np
import pandas as pd
import sklearn
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
## coefficient of determination (R**2)
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

#######################################################

N_EPOCHS = 10000
batch_size = 5
learning_rate = 0.1    ## 0.01  ## 0.1 ## 1e-5 

#######################################################

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)


In [2]:

CFD_raw_data = pd.read_csv('CFD.6.2022.csv') 


In [3]:

headers_list = CFD_raw_data.columns.values.tolist()
## print(headers_list)
## print(len(headers_list))
## print(headers_list[27])

for i, name in enumerate(headers_list):
    print((i, name))
    

(0, 'index')
(1, 'i_pul_coal_inj_kg_thm')
(2, 'i_nat_gas_inj_kg_thm')
(3, 'i_nat_gas_t_k')
(4, 'i_o2_vol_perce')
(5, 'i_bf_windrate_nm3_hr')
(6, 'i_hb_moist_g_nm3')
(7, 'i_hot_blast_press_pa')
(8, 'i_hot_blast_temp_k')
(9, 'i_coke_weight_kg')
(10, 'i_ore_weight_kg')
(11, 'o_tuyere_exit_velo_m_s')
(12, 'o_tuyere_t_k')
(13, 'o_raceway_flame_temp_k')
(14, 'o_raceway_coal_burn_perce')
(15, 'o_raceway_volume_m')
(16, 'o_raceway_depth _m')
(17, 'o_shaft_co_utiliz')
(18, 'o_shaft_h2_utiliz')
(19, 'o_shaft_top_gas_temp_c')
(20, 'o_shaft_press_drop_pa')
(21, 'o_shaft_coke_rate_kg_thm')
(22, 'o_shaft_cohesive_zone_tip_height_m')
(23, 'o_shaft_cohes_zone_root_height_m')
(24, 'o_shaft_co_v_perc')
(25, 'o_shaft_co2_v_perc')
(26, 'o_shaft_h2_v_perce')
(27, 'o_shaft_n2_v_perc')


In [4]:

CFDdata_np = CFD_raw_data.to_numpy()

## print(CFDdata_np)
print(CFDdata_np.shape)


(56, 28)


In [5]:

#######################################################

input_indeces  = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
output_indeces = [11, 12 ,13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

#######################################################

X = CFDdata_np[:, input_indeces]

y = CFDdata_np[:, output_indeces]

print(X.shape)
print(y.shape)

#######################################################

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#######################################################

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



(56, 10)
(56, 17)
(44, 10)
(12, 10)
(44, 17)
(12, 17)


In [6]:

## fix data type
X_train  = X_train.astype(np.float32)
X_test   = X_test.astype(np.float32)
y_train  = y_train.astype(np.float32)
y_test   = y_test.astype(np.float32)



##################################################

'''

ss_X = StandardScaler()
obj_X = ss_X.fit(X_train)
X_train = obj_X.transform(X_train)
X_test  = obj_X.transform(X_test)

ss_y = StandardScaler()
obj_y = ss_y.fit(y_train)
y_train = obj_y.transform(y_train)
y_test  = obj_y.transform(y_test)

'''

##################################################



'\n\nss_X = StandardScaler()\nobj_X = ss_X.fit(X_train)\nX_train = obj_X.transform(X_train)\nX_test  = obj_X.transform(X_test)\n\nss_y = StandardScaler()\nobj_y = ss_y.fit(y_train)\ny_train = obj_y.transform(y_train)\ny_test  = obj_y.transform(y_test)\n\n'

In [7]:

X_train_tr  = torch.from_numpy(X_train)
X_test_tr   = torch.from_numpy(X_test)
y_train_tr  = torch.from_numpy(y_train)
y_test_tr   = torch.from_numpy(y_test)


In [8]:

###################################################################
## for scaling

epsilon = 0.0001

x_means      =  X_train_tr.mean(0, keepdim=True)
x_deviations =  X_train_tr.std(0, keepdim=True) + epsilon

X_train_tr_scaled = (X_train_tr - x_means) / x_deviations
X_test_tr_scaled  = (X_test_tr  - x_means) / x_deviations

###################################################################
## for multiple regression outputs, normalize the y tensors too? 
## usually this is not done, but testing here

y_means      = y_train_tr.mean(0, keepdim=True)
y_deviations = y_train_tr.std(0,  keepdim=True) + epsilon

y_train_tr_scaled = (y_train_tr - y_means) / y_deviations
y_test_tr_scaled  = (y_test_tr - y_means) / y_deviations

####################################################################

print(X_train_tr.shape)
print(X_test_tr.shape)
print(y_train_tr.shape)
print(y_test_tr.shape)

print(X_train_tr_scaled.shape)
print(X_test_tr_scaled.shape)
print(y_train_tr_scaled.shape)
print(y_test_tr_scaled.shape)



torch.Size([44, 10])
torch.Size([12, 10])
torch.Size([44, 17])
torch.Size([12, 17])
torch.Size([44, 10])
torch.Size([12, 10])
torch.Size([44, 17])
torch.Size([12, 17])


In [9]:

#######################################################
## define dataset

train_ds = TensorDataset(X_train_tr, y_train_tr_scaled)

#######################################################
## define dataloader

train_dl = DataLoader(train_ds, batch_size, shuffle=True)


#######################################################


## Deep Learning Architecture

This DL model has 2 outputs. One output for the scaled y values and one for the descaled y values. 

In [10]:
    
#############################################################


class DL_Net_Dropout(nn.Module):
    
    ## initialize the layers
    def __init__(self, x_means, x_deviations, y_means, y_deviations):
        super().__init__()
        
        self.x_means      = x_means
        self.x_deviations = x_deviations
        self.y_means      = y_means
        self.y_deviations = y_deviations
        
        self.linear1 = nn.Linear(10, 32)
        self.act1    = nn.Tanh() 
        self.linear2 = nn.Linear(32, 24)
        self.act2    = nn.Tanh() 
        self.linear3 = nn.Linear(24, 17)
        self.dropout = nn.Dropout(0.25)
    
    ## perform inference
    def forward(self, x):
        
        x = (x - self.x_means) / self.x_deviations
        x = self.linear1(x)
        x = self.act1(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.act2(x)
        y = self.dropout(x)
        
        y_scaled = self.linear3(y)
        y_descaled = y_scaled * self.y_deviations + self.y_means
        
        return y_descaled, y_scaled

      
#############################################################


In [11]:

#######################################################


def fit(num_epochs, model, loss_fn, opt):
    
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            pred_descaled, pred_scaled = model(xb)
            loss = loss_fn(pred_scaled, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            ## print('Training loss:', loss_fn(pred_scaled, yb))
            
    pred_descaled, pred_scaled = model(X_train_tr)
    print('Training loss:', loss_fn(  pred_scaled,                   y_train_tr_scaled          ))
    print('Training R**2:', r2_score( pred_scaled.detach().numpy(),  y_train_tr_scaled.numpy()  ))
    

#######################################################


In [12]:

model = DL_Net_Dropout(x_means, x_deviations, y_means, y_deviations)
opt = torch.optim.Adam(   model.parameters(), lr=learning_rate   )
loss_fn = F.mse_loss

## loss_fn = F.l1_loss



In [13]:

fit(N_EPOCHS, model, loss_fn, opt)


Training loss: tensor(1.2911, grad_fn=<MseLossBackward0>)
Training R**2: -8.200988553141734


## Predict outputs

In [14]:

for i in range(len(X_test_tr)):
    print("**************************************************")
    print("preds, real")
    preds_descaled, preds_scaled = model(X_test_tr[i])
    ## print(  preds_descaled.shape  )
    ## print(  preds_descaled  )
    np_pred = preds_descaled[0].detach().numpy()              ## [0]
    np_real = y_test_tr[i].detach().numpy()
    ## print(np_pred.shape)
    ## print(np_real.shape)

    for j in range(len(np_pred)):
        print((np_pred[j], np_real[j]))
    



**************************************************
preds, real
(131.32228, 133.4865)
(1420.3427, 1440.96)
(2411.1772, 2202.8)
(66.500114, 0.0)
(0.21077059, 0.1965359)
(0.8303129, 0.8111)
(46.27024, 47.20085)
(44.43135, 42.99392)
(98.423325, 118.8785)
(113932.47, 108434.33)
(474.8391, 462.73654)
(32.64458, 31.83963)
(9.912773, 10.649022)
(0.2489319, 0.2171919)
(0.21671897, 0.1941631)
(0.04139036, 0.043832704)
(0.4962294, 0.5448123)
**************************************************
preds, real
(129.17506, 133.9054)
(1423.3619, 1444.46)
(2326.02, 2214.316)
(68.69281, 0.0)
(0.21206772, 0.1965359)
(0.8311855, 0.8111)
(46.338833, 48.77197)
(44.84702, 43.93872)
(113.28984, 104.5031)
(112501.31, 101647.98)
(454.9361, 454.2938)
(32.6291, 31.75963)
(10.039142, 9.713824)
(0.23577124, 0.2119294)
(0.20564713, 0.2017688)
(0.0337026, 0.043137867)
(0.52663064, 0.543164)
**************************************************
preds, real
(130.76437, 157.34734)
(1434.958, 1367.4567)
(2356.5642, 1805.758)
(4

In [15]:

pred_descaled, pred_scaled = model(X_test_tr)
print('Test loss:', loss_fn(     pred_scaled,                   y_test_tr_scaled          ))
print('Testing R**2:', r2_score( pred_scaled.detach().numpy(),  y_test_tr_scaled.numpy()  ))


Test loss: tensor(1.1411, grad_fn=<MseLossBackward0>)
Testing R**2: -5.066213619133827


In [16]:

'''

model.eval()

dummy_input = torch.randn(1, 10)


input_names = ["input1"]
output_names = ["output1", "output2"]

torch.onnx.export(
  model, 
  dummy_input, 
  "ONNXmodels/CFDallNORM.onnx", 
  verbose=False, 
  input_names  = input_names,
  output_names = output_names
)

'''


'\n\nmodel.eval()\n\ndummy_input = torch.randn(1, 10)\n\n\ninput_names = ["input1"]\noutput_names = ["output1", "output2"]\n\ntorch.onnx.export(\n  model, \n  dummy_input, \n  "ONNXmodels/CFDallNORM.onnx", \n  verbose=False, \n  input_names  = input_names,\n  output_names = output_names\n)\n\n'